Importar Módulos

In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import win32com.client as win32

Ler os arquivos excel e realizar tratamento de dados

In [ ]:
produtos_df = pd.read_excel(r'Produtos.xlsx')
produtos_df = produtos_df.fillna('-')
display(produtos_df)

Definindo função para tratar preços dos sites

In [ ]:
def tratar_texto(texto):
    return float(texto.replace('R$', '').replace(' ', '').replace('.', '').replace(',', '.'))

Realizando a busca e salvando preços mais baixos na planilha do Excel

In [ ]:
driver = webdriver.Chrome()
driver.set_window_position(-10000, 0)

enviar_email = False
desconto_min = 0.2

for i, linha in produtos_df.iterrows():
    driver.get(linha['Amazon'])

    try:
        preco_amazon = driver.find_element(By.CLASS_NAME, 'a-price-whole').text
        preco_amazon = tratar_texto(preco_amazon)
    except:
        print(f'Produto {linha["Produtos"]} não disponível na Amazon')
        preco_amazon = linha['Preço Original'] * 3

    driver.get(linha['Magazine Luiza'])

    try:
        preco_magalu = driver.find_element(By.XPATH, '//*[@id="__next"]/div/main/section[4]/div[4]/div[1]/div/div/p[2]').text
        preco_magalu = tratar_texto(preco_magalu)
    except:
        print(f'Produto {linha["Produtos"]} não disponível na Magazine Luiza')
        preco_magalu = linha['Preço Original'] * 3
    
    preco_original = linha['Preço Original']
    
    lista_precos =[(preco_amazon, 'Amazon'), (preco_magalu, 'Magazine Luiza'), (preco_original, 'Preço Original')]
    lista_precos.sort()
    
    produtos_df.loc[i, 'Preço Atual'] = lista_precos[0][0]
    produtos_df.loc[i, 'Local'] = lista_precos[0][1]

    if lista_precos[0][0] <= preco_original * (1 - desconto_min):
        enviar_email = True
    
produtos_df.to_excel('Produtos.xlsx')

driver.quit()

Enviando e-mail com as informações de desconto (se houver)

In [ ]:
outlook = win32.Dispatch('outlook.application')

if enviar_email:
    mail = outlook.CreateItem(0)
    mail.To = 'youremail@gmail.com'
    mail.Subject = f'Produto(s) encontrado(s) com mais de {desconto_min:.0%}% de Desconto'
    
    df_filtrada = produtos_df.loc[produtos_df['Preço Atual'] <= produtos_df['Preço Original'] * (1-desconto_min), :]
    
    mail.HTMLBody =  f'<p>Esses sãos os produtos com mais de {desconto_min:.0%}% de desconto</p>{df_filtrada.to_html()}'
    
    
    mail.Send()